# DDD.jl

This is a tutorial and demo of `DDD.jl`.

We start by loading `DDD`. We also load the standard library `Plots` for plotting and `LinearAlgebra` for cross products. We want to make interactive plots via `PlotlyJS` but it's not a dependancy of `DDD.jl` so we have to add it to the notebook.

In [ ]:
using DDD, Plots, LinearAlgebra, Pkg
Pkg.add("PlotlyJS")
plotlyjs()

## Parameters

After loading the package, we need to set up some parameters that control all aspects of the systems we will be simulating.

There are various ways of creating these, but here we use named constructors because they perform validations and automatically calculate derived values. 

We could also create them manually without validation by using the intrinsic structure constructors, or override the defaults on the named constructor. We can also load parameters using `JSON` or `jld2` files, which we show later.

!!! note 
    In interactive mode, Julia displays the output of its commands. The structures are rather large and the display will clutter the terminal, so we supress them by adding a `;` to the end of each command. This is only necessary when working in the REPL or notebooks.

In [ ]:
dlnParams = DislocationParameters(;
    mobility = mobBCC(),
    dragCoeffs = (edge = 1, screw = 1e-1, climb = 1e9, line = 1e-5),
    coreRad = 0.015 * sqrt(2),
    minSegLen = 0.15 * sqrt(2),
    maxSegLen = 1.5 * sqrt(2),
    coreEnergy = 1 / (4 * π) * log(0.015 * sqrt(2) / 3.5e-5),
    coreRadMag = 3.5e-4,
);

matParams = MaterialParameters(; crystalStruct = BCC(), μ = 1, μMag = 80e3, ν = 0.25);

femParamsC = FEMParameters(;
    type = DispatchRegularCuboidMesh(),
    order = LinearElement(),
    model = CantileverLoad(),
    dx = 23.0,
    dy = 17.0,
    dz = 13.0,
    mx = 7,
    my = 5,
    mz = 3,
);

slipSystems = SlipSystem(;
    crystalStruct = BCC(),
    slipPlane = Float64[-1 1; 1 -1; 0 0],
    bVec = Float64[1 1; 1 1; 1 -1],
);

intParams = IntegrationParameters(;
    method = AdaptiveEulerTrapezoid(),
    abstol = dlnParams.collisionDist / 2,
    reltol = dlnParams.collisionDist / 2,
);

intTime = IntegrationTime(; dt = 0.0, time = 0.0);

We can see the fields of these structures by using the `fieldnames()` function on their types.

In [ ]:
# On the type of the variable.
fieldnames(typeof(matParams))

In [ ]:
# Directly on the type itself.
fieldnames(DislocationParameters)

We can also browse the documentation for additional information. Say we want to find out more about the function `DislocationParameters`.

In [ ]:
?DislocationParameters

`?` is used to browse for the documentation of types and functions. When used on variables, it provides type information.

In [ ]:
?femParamsC

Note how the types of all fields are reported back. The summary also provides the type heirarchy of the variable. This information allows the compiler to optimise code and enables one of the most powerful features of Julia, multiple dispatch.

The fields in a structure can be accessed via the dot syntax in the same way it's done in Python, Matlab and C.

Julia draws heavy inspiration from functiional languages, so a lot of its syntax is syntactic sugar for functions. For example, accessing fields via dot syntax is syntactic sugar for `getproperty()`. Other examples of syntactic sugar include accessing and setting the values of array entries. 

In [ ]:
femParamsC.model

`CantileverLoad()` is a concrete type. It is used to set the appropriate boundary conditions by using multiple dispatch later on.

## Mesh generation

With all the parameters in place, we can start generating the data to be used in the simulation. The variable `meshC` contains all the information regarding the mesh. The constructor uses the canonical node numbering to minimise the need for custom code.

In [ ]:
meshC = buildMesh(matParams, femParamsC);

If you're curious as to why we're suppressing the display output on all these calls run the following two cells. The first displays the structure with its content, the second only the type information.

In [ ]:
meshC

In [ ]:
?meshC

One might think all the verbosity is a bad thing, not so. It means the compiler knows exactly the types of each element. In some cases, it even knows the size of some arrays. The power of Julia comes from this robust typing system, which lets the compiler agressively optimise code.

A neat feature of Julia is that visualisation of large sparse matrices is made very easy via the REPL. We can visualise the stiffness matrix of this mesh on the REPL by simply accessing it. This is equivalent to using `spy()`, but is built into the display functionality.

In [ ]:
meshC.K

It even works on slices.

In [ ]:
meshC.K[1:200, 1:200]

## Boundary conditions

Julia has a few plotting libraries, but one of the most robust and developed is `Plots.jl`. It offers multiple backends for multiple needs under a singular interface thanks to multiple dispatch. Here we will use `plotlyjs()` because it generates nice, interactive plots with great 3D capabilities. Matlab's own plotting functionality is based on `PlotlyJS`, so the feel will be familiar to anyone coming from Matlab.

Try double clicking on the label. It lets you select different series. The mesh is created using canonical numberings, so the labels correspond to them (except corners which are lumped together).

In [ ]:
using Plots, Pkg
Pkg.add("PlotlyJS")
plotlyjs()

In [ ]:
figFE = plotFEDomain(meshC)

With the `meshC` and `femParamsC`, we can define the boundary conditions for modelling a cantilever loading experiment.

In [ ]:
boundaryC, forceDispC = Boundaries(femParamsC, meshC);

The `Boundaries()` function accepts keyword arguments that let the user modify the boundary nodes. However, we've used the defaults.

We can also plot these to make sure we're doing it right.

In [ ]:
figCBC = plotBoundaries(boundaryC, meshC)

We've also implemented boundary conditions for pillar loading, though they are yet to be tested.

In [ ]:
femParamsP = FEMParameters(;
    type = DispatchRegularCuboidMesh(),
    order = LinearElement(),
    model = PillarLoad(),
    dx = 17.0,
    dy = 13.0,
    dz = 23.0,
    mx = 5,
    my = 3,
    mz = 7,
);
meshP = buildMesh(matParams, femParamsP);
boundaryP, forceDispP = Boundaries(femParamsP, meshP);
figPBC = plotBoundaries(boundaryP, meshP)

## Dislocation generation

We could have created our dislocations and then the FE mesh, but we want to use the mesh parameters to generate the dislocations inside the domain (mostly).

In [ ]:
# Unpack the dimensions and calculate the segment length.
dx, dy, dz = femParamsC.dx, femParamsC.dy, femParamsC.dz
segLen = (dlnParams.minSegLen + dlnParams.maxSegLen) / 2

There are multiple ways of creating a dislocation network.
1. Manually, which means we also have to generate the connectivity and segment indices
    1. via the named constructor, which performs validations;
    2. via the standard constructor, which does not perform any validation.
2. Using dislocation loops, which is the preferred method as it performs validations and automatically generates everything else the network we needs.

We can create prismatic and shear loops using the `loopPrism()` and `loopShear()` types.

They will generate 5 octagonal prismatic loops in the first slip system and 3 pentagonal shear loops in the second.

In [ ]:
# Prismatic loop with 8 sides, one node per side. 
# When used to generate a network will produce 5 loops.
prismOct = DislocationLoop(;
    loopType = loopPrism(), # Prismatic loop
    numSides = 8,   # 8 Sides
    nodeSide = 1,   # One node per side
    numLoops = 5,   # When used to generate a network will create 5 loops
    segLen = segLen * ones(8), # Every segment will have the same length (regular octahedron)
    slipSystemIdx = 1,  # Belongs to the first slip system
    slipSystem = slipSystems, # Slip systems
    label = nodeTypeDln.(ones(Int, 8)), # Node labels, dictate what type of node it is
    buffer = 0, # Spacing buffer for distribution in the domain
    range = [0 dx; 0 dy; 0 dz], # Range of the distribution
    dist = Rand(), # They will be randomly distributed in the domain
);

shearPent = DislocationLoop(;
    loopType = loopShear(), # Shear loop
    numSides = 5,   # 5 Sides
    nodeSide = 2,   # Two nodes per side (corner and middle of segments)
    numLoops = 3,
    segLen = segLen * ones(10),
    slipSystemIdx = 2,
    slipSystem = slipSystems,
    label = nodeTypeDln.(ones(Int, 10)),
    buffer = 0,
    range = [0 dx; 0 dy; 0 dz],
    dist = Rand(),
);

The type `nodeTypeDln` is an enumerated type. These types can only take on specific values. They provide safety and efficiency when using categorial variables.

In 3D discrete dislocation dynamics we can define nodes as having certain characteristics. We use node labels to enforce this behaviour. 

!!! note
    There is also `nodeTypeFE` but is not currently used and may be removed in the future.

In [ ]:
?nodeTypeDln

We can visualise the loops on their own. They are generated centered at the origin.

!!! note
    In Julia, the convention to append a `!` at the end of a function means the function modifies its input.

In [ ]:
dlnFig = plotNodes(
    prismOct,
    m = 2,
    l = 3,
    linecolor = :orange,
    markershape = :circle,
    markercolor = :orange,
    legend = false,
)
plotNodes!(
    dlnFig,
    shearPent,
    m = 2,
    l = 3,
    linecolor = :purple,
    markershape = :square,
    markercolor = :purple,
    legend = false,
)

We can now use these loops to create the network.

!!! note
    We use a tuple of loops `(,)` but one can also use vectors `[]`, a single loop or an iterator that returns loops.

In [ ]:
network = DislocationNetwork((prismOct, shearPent));

We can visualise the network on its own, without the FE domain.

In [ ]:
networkFig = plotNodes(
    network,
    m = 2,
    l = 3,
    linecolor = :blue,
    markershape = :circle,
    markercolor = :blue,
    legend = false,
)

Or we can visualise the network with the FE domain around it.

In [ ]:
networkFEFig = plotNodes(
    meshC,
    network,
    m = 2,
    l = 3,
    linecolor = :blue,
    markershape = :circle,
    markercolor = :blue,
    legend = false,
)

Using the convention for mutating functions we can add more dislocation loops to the network. Here we're adding more prismatic octagons.

In [ ]:
network = DislocationNetwork!(network, prismOct);

Which we can also plot.

In [ ]:
networkFEFig = plotNodes(
    meshC,
    network,
    m = 2,
    l = 3,
    linecolor = :blue,
    markershape = :circle,
    markercolor = :blue,
    legend = false,
)

## Current capabilities

The code comes with a fairly exhaustive set of tests.

Features marked with a tick have been thoroughly tested.

- [x] IO
    - [x] JSON
    - [x] jld2
- [ ] Post-processing
    - [ ] Plotting
        - [x] Loops
        - [x] Network
        - [x] FE domain
        - [ ] Boundary conditions
- [x] Forces
    - [x] Peach-Köhler in a regular mesh with linear hexahedral elements
    - [x] Self-forces
    - [x] O(N^2) remote forces
    - [x] Total forces
- [ ] Topology
    - [x] Internal node remeshing
    - [x] Surface remeshing
    - [ ] Virtual remeshing
    - [ ] Collisions
        - [x] Collision detection
        - [ ] Collision resolution
- [ ] Linear hexahedral FEM
    - [x] Meshing
    - [ ] Shape functions
        - [ ] 2D
        - [x] 3D
    - [ ] Gauss quadrature
        - [x] 1D
        - [ ] 2D
        - [ ] 3D
    - [ ] Boundary conditions
        - [x] Cantilever loading
        - [ ] Pillar loading
- [x] Dislocation-induced displacements
- [ ] Dislocation-induced tractions
    - [x] Field point stresses
    - [ ] Lazy numeric tractions q = 1
- [x] Mobility
    - [x] Outdated BCC mobility law
- [ ] Time integration
    - [ ] Adaptive euler trapezoid
    - [ ] Surface velocities

We're not going to provide an extensive showing of the tests, for information on test coverage click on either of these badges.

[![Codecov](https://codecov.io/gh/dcelisgarza/DDD.jl/branch/master/graph/badge.svg)](https://codecov.io/gh/dcelisgarza/DDD.jl)
[![Coveralls](https://coveralls.io/repos/github/dcelisgarza/DDD.jl/badge.svg?branch=master)](https://coveralls.io/github/dcelisgarza/DDD.jl?branch=master)

However we do want to show how the code can be used.

### Forces

The forces can be calculated for the whole network or specific nodes/collections of nodes.

!!! note
    Julia has unicode support, so we use `\approx + Tab` to use `≈` instead of the `isapprox()` function. We don't do a strict comparison because floating point arithmetic is in general non-commutative.

!!! warning
    We have not yet implemented filtering of virtual segments from force calculations. It should not be difficult.

In [ ]:
# Peach-Köhler force. We don't have a function to apply loading yet so we just generate a random applied stress.
forceDispC.uHat .= rand(-50:50, length(forceDispC.uHat)) .* rand(length(forceDispC.uHat))
fPK = calcPKForce(meshC, forceDispC, network)
fPK69 = calcPKForce(meshC, forceDispC, network, 69)
fPK420 = calcPKForce(meshC, forceDispC, network, [4, 20])
println("fPK[:, 69] ≈ fPK69 : ", fPK[:, 69] ≈ fPK69)
println("fPK[:, [4, 20]] ≈ fPK420 : ", fPK[:, [4, 20]] ≈ fPK420)

In [ ]:
sf = calcSelfForce(dlnParams, matParams, network)
sf42 = calcSelfForce(dlnParams, matParams, network, 42)
sf619 = calcSelfForce(dlnParams, matParams, network, [6, 1, 9])
println("sf[1][:, 42] ≈ sf42[1] && sf[2][:, 42] ≈ sf42[2] : ",
        sf[1][:, 42] ≈ sf42[1] && sf[2][:, 42] ≈ sf42[2])
println("sf[1][:, [6, 1, 9]] ≈ sf619[1] && sf[2][:, [6, 1, 9]] ≈ sf619[2] : ",
        sf[1][:, [6, 1, 9]] ≈ sf619[1] && sf[2][:, [6, 1, 9]] ≈ sf619[2])

In [ ]:
# Remote forces.
rf = calcSegSegForce(dlnParams, matParams, network)
rf13 = calcSegSegForce(dlnParams, matParams, network, 13)
rf666 = calcSegSegForce(dlnParams, matParams, network, [66, 6])
println("rf[:, :, 13] ≈ rf13 : ", rf[:, :, 13] ≈ rf13)
println("rf[:, :, [66, 6]] ≈ rf666 : ", rf[:, :, [66, 6]] ≈ rf666)

In [ ]:
# Total forces.
f = calcSegForce(dlnParams, matParams, meshC, forceDispC, network)
f57 = calcSegForce(dlnParams, matParams, meshC, forceDispC, network, 57)
f316 = calcSegForce(
    dlnParams,
    matParams,
    meshC,
    forceDispC,
    network,
    [31, 6],
)
println("f[:, :, 57] ≈ f57 : ", f[:, :, 57] ≈ f57)
println("f[:, :, [31, 6]] ≈ f316 : ", f[:, :, [31, 6]] ≈ f316)

All of these functions have in-place versions that modify the network directly so they don't allocate as much memory. 

We only show the total force calculation because it aggregates the forces in the network.

!!! note
    In order to take advantage of immutable structures' perfromance benefits we use 1-element arrays to keep track of the number of segments and nodes.

In [ ]:
calcSegForce!(dlnParams, matParams, meshC, forceDispC, network)
println("f ≈ network.segForce[:, :, 1:network.numSeg[1]] : ", f ≈ network.segForce[:, :, 1:network.numSeg[1]])

### Mobility

We have an outdated mobility law that has also been tested, however this will change in the future.

In [ ]:
nodeForce, nodeVel = dlnMobility(dlnParams, matParams, network)
dlnMobility!(dlnParams, matParams, network)
nodeVel ≈ network.nodeVel[:, 1:network.numNode[1]]
nodeForce ≈ network.nodeForce[:, 1:network.numNode[1]]

### Remeshing

We can coarsen and refine the the internal network, however since we used the default parameters nothing will happen.

!!! note
    If instead you make `minArea = coreRad^2 / sqrt(2)`,
        ```julia
        dlnParams = DislocationParameters(;
            mobility = mobBCC(),
            dragCoeffs = (edge = 1, screw = 1e-1, climb = 1e9, line = 1e-5),
            coreRad = 0.015 * sqrt(2),
            minSegLen = 0.15 * sqrt(2),
            maxSegLen = 1.5 * sqrt(2),
            coreRad = 0.015^2 * sqrt(2),
            coreEnergy = 1 / (4 * π) * log(0.015 * sqrt(2) / 3.5e-5),
            coreRadMag = 3.5e-4,
        );
        ```
    and refine the network, it will generate nodes in the middle of every segment.

In [ ]:
# Remove links that are too small or make small, vanishing hinges.
network = coarsenNetwork!(dlnParams, matParams, meshC, forceDispC, network);
# Refine adds nodes to segments that are too long or hinges with sides too large.
network = refineNetwork!(dlnParams, matParams, meshC, forceDispC, network);
networkFEFig = plotNodes(
    meshC,
    network,
    m = 2,
    l = 3,
    linecolor = :blue,
    markershape = :circle,
    markercolor = :blue,
    legend = false,
)

We can also remesh the surface, if there are any segments poking out of the domain they will be remeshed such that a node is placed on the surface and the other node is projected out.

This is because the displacement calculation uses triangles. However, we also have to prevent dislocations from exiting the fixed end. The impenetrable faces are defined by `Boundaries()`.

The surface remeshing uses nodal velocities to find the intersect with the volume, so we need to make sure we have calculated the forces and nodal velocities beforehand.

In [ ]:
# Uncomment following 3 lines to add more loops to network.
# network = DislocationNetwork!(network, (prismOct, shearPent))
# calcSegForce!(dlnParams, matParams, meshC, forceDispC, network)
# dlnMobility!(dlnParams, matParams, network)

# Remesh the surface.
network = remeshSurfaceNetwork!(meshC, boundaryC, network);
# Coarsen the internal network.
network = coarsenNetwork!(dlnParams, matParams, meshC, forceDispC, network);
# Refine the internal network.
network = refineNetwork!(dlnParams, matParams, meshC, forceDispC, network);

You can keep generating networks or adding more dislocations using `DislocationNetwork!()` as previously shown. 

In this case, the impenetrable surface is the `yz` plane at `x = 0`, so any segments that exit from there will be removed and an immovable node will be generated on the surface. Dislocations that exit from other surfaces will be projected out along the face normal of the face it exited out from. They will be projected and the FEM scale, both found in `meshC`. The projection is robust enough to work on corners and edges.

In [ ]:
remeshedFig = plotNodes(
    meshC,
    network,
    m = 2,
    l = 3,
    linecolor = :blue,
    markershape = :circle,
    markercolor = :blue,
    legend = false,
)

If you would like to see how the nodes are projected run the following cell.

In [ ]:
using StaticArrays

function plotNodesProj(mesh::AbstractMesh, network::DislocationNetwork, args...; kw...)
    label = network.label
    coord = network.coord
    links = network.links
    numNode = network.numNode[1]
    elemT = eltype(links)
    fig = plot()
    for i in 1:numNode
        n1n2 = SVector{2, elemT}(links[1, i], links[2, i])
        plot!(fig, coord[1, n1n2], coord[2, n1n2], coord[3, n1n2], args...; kw...)
    end

    # Plot FEM domain.
    vertices = reshape(collect(Iterators.flatten(mesh.vertices.vertices)), 3, :)
    vertices = reshape(collect(Iterators.flatten(mesh.vertices.vertices)), 3, :)
    face1 = [mesh.faces[:, 5]; mesh.faces[1, 5]]
    face2 = [mesh.faces[:, 6]; mesh.faces[1, 6]]
    surf1 = vertices[:, face1]
    surf2 = vertices[:, face2]
    plot!(fig, surf1[1, :], surf1[2, :], surf1[3, :], linecolor = :black, linewidth = 2)
    plot!(fig, surf2[1, :], surf2[2, :], surf2[3, :], linecolor = :black, linewidth = 2)
    side = vertices[:, [1, 5]]
    plot!(fig, side[1, :], side[2, :], side[3, :], linecolor = :black, linewidth = 2)
    side = vertices[:, [2, 6]]
    plot!(fig, side[1, :], side[2, :], side[3, :], linecolor = :black, linewidth = 2)
    side = vertices[:, [3, 7]]
    plot!(fig, side[1, :], side[2, :], side[3, :], linecolor = :black, linewidth = 2)
    side = vertices[:, [4, 8]]
    plot!(fig, side[1, :], side[2, :], side[3, :], linecolor = :black, linewidth = 2)

    xlims = (
        minimum(vertices[1, :]) - 0.5 * maximum(vertices[1, :]),
        maximum(vertices[1, :]) + 0.5 * maximum(vertices[1, :]),
    )
    ylims = (
        minimum(vertices[2, :]) - 0.5 * maximum(vertices[2, :]),
        maximum(vertices[2, :]) + 0.5 * maximum(vertices[2, :]),
    )
    zlims = (
        minimum(vertices[3, :]) - 0.5 * maximum(vertices[3, :]),
        maximum(vertices[3, :]) + 0.5 * maximum(vertices[1, :]),
    )
    plot!(fig, xlims = xlims, ylims = ylims, zlims = zlims)
    plot!(fig; xlabel = "x", ylabel = "y", zlabel = "z")

    return fig
end

remeshedFig = plotNodesProj(
    meshC,
    network,
    m = 2,
    l = 3,
    linecolor = :blue,
    markershape = :circle,
    markercolor = :blue,
    legend = false,
)

### Collisions

So far we've implemented collision detection and resolution. However, collision resolution is still untested. The collision detection algorithm returns whether a collision is imminent.

In [ ]:
#=
    collision == true if a collision was found false otherwise.
    collisionType == :null if no collision was found, 
        :hinge if two segments sharing a node collide,
        :twoLine if two unconnected segments collide.
    n1s1 == node 1 segment 1, n2s1 == node 2 segment 1
    n1s2 == node 1 segment 2, n2s2 == node 2 segment 2
    s1 == segment 1, s2 == segment 2
    L1, L2 == normalised position along the segments where collision will occur
=#
collision, collisionType, n1s1, n2s1, n1s2, n2s2, s1, s2, L1, L2 = 
    detectCollision(dlnParams, network)

### Dislocation-induced displacements

The code also has the capability of calculating dislocation induced displacements by Bromage and Tarleton [[1]](https://iopscience.iop.org/article/10.1088/1361-651X/aae404/pdf).

In [ ]:
# Dislocation displacements on the displacement nodes.
uTilde = calc_uTilde(meshC, boundaryC, matParams, network)
calc_uTilde!(forceDispC, meshC, boundaryC, matParams, network)
uTilde ≈ forceDispC.uTilde[boundaryC.uDofsDln]

### Lazy numeric tractions

We can also calculate lazy numeric tractions for 1 gauss quadrature point directly on finite element nodes.

In [ ]:
N, A = getTGammaDlnNormsArea(boundaryC, meshC)
points = meshC.coord[:, boundaryC.tGammaDln]
σTilde = calc_σTilde(points, dlnParams, matParams, network)
T = calcNumericTractions(σTilde, N, A)

### Dislocation-induced stresses

We can also calculate dislocation stress fields. Starting with a screw dislocation.

!!! note
    We have to manually create the dislocation network to generate a straight dislocation line so we have to call `makeConnect!(network)` and `getSegmentIdx!(network)` to generate the connectivity matrix and segment indices.

In [ ]:
numNode = 102;
numSeg = numNode - 1;
len = numNode + 1;
xrange = range(-100, 100, length = len);
yrange = range(-100, 100, length = len);

X = ones(length(yrange)) .* xrange';
Y = ones(length(xrange))' .* yrange;
Z = zeros(length(xrange))' .* zeros(len);
points = [X[:]'; Y[:]'; Z[:]'];
X, Y, Z = nothing, nothing, nothing;

a = 5 * norm(b)

# We need some dummy parameters.
matParams =
    MaterialParameters(; crystalStruct = BCC(), μ = 1.0, μMag = 1.0, ν = 0.28);
dlnParams = DislocationParameters(;
    coreRad = a,
    coreRadMag = 1.0,
    minSegLen = a + 2,
    maxSegLen = a + 3,
    minArea = a + 1,
    maxArea = a + 2,
    mobility = mobBCC(),
);

In [ ]:
# Screw dislocation.

l = Float64[0; 0; 1]
b = Float64[0; 0; 1]
n = b × l

links = zeros(Int, 2, numSeg)
slipPlane = zeros(3, numSeg)
bVec = zeros(3, numSeg)
coord = [zeros(len)'; zeros(len)'; xrange']
label = zeros(nodeTypeDln, len)
nodeVel = similar(coord)
nodeForce = similar(coord)
for i in 1:numSeg
    links[:, i] .= (i, i + 1)
    slipPlane[:, i] = n
    bVec[:, i] = b
end

network = DislocationNetwork(;
    links = links,
    slipPlane = slipPlane,
    bVec = bVec,
    coord = coord,
    label = label,
    nodeVel = nodeVel,
    nodeForce = nodeForce,
)
makeConnect!(network)
getSegmentIdx!(network)

# Out of place calculation.
stress = reshape(calc_σTilde(points, dlnParams, matParams, network), 6, len, :)

σ = zeros(6, size(points, 2))
# In-place calculation.
calc_σTilde!(σ, points, dlnParams, matParams, network)
σ = reshape(σ, 6, len, :)

println("σ ≈ stress : ", σ ≈ stress)

figXY = contourf(xrange, yrange, σ[5, :, :], levels = 30)
figXZ = contourf(xrange, yrange, σ[6, :, :], levels = 30)


And now an edge dislocation.

In [ ]:
# Edge dislocation.

l = Float64[0; 0; 1]
b = Float64[1; 0; 0]
n = b × l

links = zeros(Int, 2, numSeg)
slipPlane = zeros(3, numSeg)
bVec = zeros(3, numSeg)
coord = [zeros(len)'; zeros(len)'; xrange']
label = zeros(nodeTypeDln, len)
nodeVel = similar(coord)
nodeForce = similar(coord)
for i in 1:numSeg
    links[:, i] .= (i, i + 1)
    slipPlane[:, i] = n
    bVec[:, i] = b
end

network = DislocationNetwork(;
    links = links,
    slipPlane = slipPlane,
    bVec = bVec,
    coord = coord,
    label = label,
    nodeVel = nodeVel,
    nodeForce = nodeForce,
)
makeConnect!(network)
getSegmentIdx!(network)

# Out of place calculation.
stress = reshape(calc_σTilde(points, dlnParams, matParams, network), 6, len, :)

σ = zeros(6, size(points, 2))
# In-place calculation.
calc_σTilde!(σ, points, dlnParams, matParams, network)
σ = reshape(σ, 6, len, :)

println("σ ≈ stress : ", σ ≈ stress)

figXX = contourf(xrange, yrange, σ[1, :, :], levels = 30)
figYY = contourf(xrange, yrange, σ[2, :, :], levels = 30)
figZZ = contourf(xrange, yrange, σ[3, :, :], levels = 30)
figXY = contourf(xrange, yrange, σ[4, :, :], levels = 30)

## Future capabilities

- [ ] Post-processing
    - [ ] Plotting
        - [ ] Plot recipes
        - [ ] Nodal displacement and slip step plotting
        - [ ] Load-displacemen
        - [ ] Strain-stress
        - [ ] gif and video generation
- [ ] Forces
    - [ ] Inclusions
    - [ ] Hydrogen
- [ ] Topology
    - [ ] Separation
- [ ] Linear hexahedral FEM
    - [ ] Boundary conditions
        - [ ] Nanoindentation
    - [ ] Load and traction application
- [ ] FEM-DDD coupling
- [ ] Dislocation-induced tractions
    - [ ] Numeric tractions
    - [ ] Analytic tractions
- [ ] Mobility
    - [ ] BCC mobility law
    - [ ] FCC mobility law
    - [ ] HCP mobility law